# Creating Portland profile with tied ballots

In [1]:
import pandas as pd

D1_df = pd.read_csv("../data/Portland_D1_raw_from_city.csv") #insert the file name that you used when you downloaded the csv
D1_df.head()

,RowNumber,BoxID,BoxPosition,BallotID,PrecinctID,BallotStyleID,PrecinctStyleName,ScanComputerName,Status,Remade,...,"Choice_122_1:City of Portland, Councilor, District 1:3:Number of Winners 3:Write-in-122:NON","Choice_122_1:City of Portland, Councilor, District 1:4:Number of Winners 3:Write-in-122:NON","Choice_122_1:City of Portland, Councilor, District 1:5:Number of Winners 3:Write-in-122:NON","Choice_122_1:City of Portland, Councilor, District 1:6:Number of Winners 3:Write-in-122:NON","Choice_50003_1:City of Portland, Councilor, District 1:1:Number of Winners 3:Uncertified Write In:NON","Choice_50003_1:City of Portland, Councilor, District 1:2:Number of Winners 3:Uncertified Write In:NON","Choice_50003_1:City of Portland, Councilor, District 1:3:Number of Winners 3:Uncertified Write In:NON","Choice_50003_1:City of Portland, Councilor, District 1:4:Number of Winners 3:Uncertified Write In:NON","Choice_50003_1:City of Portland, Councilor, District 1:5:Number of Winners 3:Uncertified Write In:NON","Choice_50003_1:City of Portland, Councilor, District 1:6:Number of Winners 3:Uncertified Write In:NON"
0,1,RCV-0001,1,RCV-0001+10003,26,3,4506-1,ScanStation6,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,RCV-0001,2,RCV-0001+10005,32,1,2804-1,ScanStation6,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,RCV-0001,3,RCV-0001+10007,53,1,3303-1,ScanStation6,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,RCV-0001,4,RCV-0001+10009,22,1,4105-1,ScanStation6,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,RCV-0001,5,RCV-0001+10011,53,1,3303-1,ScanStation6,0,0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
# stores all columns that have ranking information
rank_columns = {i:[col for col in D1_df.columns if f'{i}:Number' in col] for i in range(1,7)}
all_rank_cols = [col for col_list in rank_columns.values() for col in col_list]

The code below scrubs any voter who did not cast at least one vote, which in turn removes any voter not from district 1.

In [3]:
D1_voters_df = D1_df[D1_df[all_rank_cols].sum(axis=1) > 0].reset_index(drop=True) # just resets the index of the df

In [4]:
from tqdm.notebook import tqdm

ranking_data = {i:[-1 for _ in range(len(D1_voters_df))] for i in range(1,7)}

for voter_index, row in tqdm(D1_voters_df.iterrows(), total=D1_voters_df.shape[0]):
    for rank_position in range(1,7):
        num_votes_cast = row[rank_columns[rank_position]].sum()

        if num_votes_cast == 0:
            cast_vote = set()

        elif num_votes_cast >= 1:
            pd_series = row[rank_columns[rank_position]]
            cast_vote_column_names = pd_series[pd_series == 1].index.to_list()
            cast_votes = {column_name.split(":")[-2] for column_name in cast_vote_column_names}
            
            cast_vote = cast_votes
            
        ranking_data[rank_position][voter_index] = cast_vote



  0%|          | 0/43669 [00:00<?, ?it/s]

In [5]:
# add the new columns
for rank_position in range(1,7):
    D1_voters_df[f"Ranking_{rank_position}"] = ranking_data[rank_position]

ranking_columns = [f"Ranking_{i}" for i in range(1,7)]
votekit_df = D1_voters_df[ranking_columns]

In [6]:
votekit_df.index.name = "Ballot Index" 
votekit_df["Voter Set"] = [set()]*len(votekit_df)
votekit_df["Weight"] = [1]*len(votekit_df)

/var/folders/r_/xjlnv23j48v1p28d0tq18wcc0000gn/T/ipykernel_95861/3598555127.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  votekit_df["Voter Set"] = [set()]*len(votekit_df)
/var/folders/r_/xjlnv23j48v1p28d0tq18wcc0000gn/T/ipykernel_95861/3598555127.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  votekit_df["Weight"] = [1]*len(votekit_df)


In [7]:
votekit_df

,Ranking_1,Ranking_2,Ranking_3,Ranking_4,Ranking_5,Ranking_6,Voter Set,Weight
Ballot Index,,,,,,,,
0,{Terrence Hayes},{Loretta Smith},{Noah Ernst},{},{},{},{},1
1,{Loretta Smith},{Steph Routh},{Timur Ender},{David Linn},{Candace Avalos},{Jamie Dunphy},{},1
2,{Loretta Smith},{Steph Routh},{Timur Ender},{David Linn},{Candace Avalos},{Jamie Dunphy},{},1
3,{Michael (Mike) Sands},{Doug Clove},{Joe Furi},{Timur Ender},{Deian Salazar},{Loretta Smith},{},1
4,{Timur Ender},{Candace Avalos},{Cayle Tern},{Steph Routh},{Michael (Mike) Sands},{Jamie Dunphy},{},1
...,...,...,...,...,...,...,...,...
43664,{Loretta Smith},{Candace Avalos},{Jamie Dunphy},{Timur Ender},{},{},{},1
43665,{Loretta Smith},{},{},{},{},{},{},1
43666,{Steph Routh},{Jamie Dunphy},{},{},{},{},{},1


In [8]:
candidates = set().union(*votekit_df[[f"Ranking_{i}" for i in range(1,7)]].to_numpy().flatten())

In [9]:
from votekit.pref_profile import PreferenceProfile

pp = PreferenceProfile(df = votekit_df, contains_rankings=True, candidates= candidates, max_ranking_length=6)

In [10]:
pp

Profile contains rankings: True
Maximum ranking length: 6
Profile contains scores: None
Candidates: ('Cayle Tern', 'Doug Clove', 'Michael (Mike) Sands', 'Deian Salazar', 'Terrence Hayes', 'Noah Ernst', 'Uncertified Write In', 'Peggy Sue Owens', 'Candace Avalos', 'Write-in-120', 'Thomas Shervey', 'Timur Ender', 'Jamie Dunphy', 'Write-in-121', 'Joe Allen', 'Write-in-122', 'David Linn', 'Joe Furi', 'Loretta Smith', 'Steph Routh')
Candidates who received votes: ('Cayle Tern', 'Doug Clove', 'Jamie Dunphy', 'Michael (Mike) Sands', 'Write-in-121', 'Deian Salazar', 'Joe Allen', 'Write-in-122', 'David Linn', 'Terrence Hayes', 'Noah Ernst', 'Joe Furi', 'Uncertified Write In', 'Peggy Sue Owens', 'Loretta Smith', 'Steph Routh', 'Candace Avalos', 'Write-in-120', 'Thomas Shervey', 'Timur Ender')
Total number of Ballot objects: 43669
Total weight of Ballot objects: 43669

In [ ]:
from votekit.cleaning import remove_and_condense, remove_repeated_candidates, condense_profile, clean_profile
from votekit.ballot import Ballot

pp = remove_and_condense(['Write-in-120', 'Write-in-121', 'Write-in-122'], pp)
pp = condense_profile(remove_repeated_candidates(pp))

# need to address true overvoting, which is listing more than 6 candidates in one slot or total
# ranking truncates when it would go over 6 candidates
def remove_overvotes(ballot):
    new_ranking = []
    num_cands = 0
    for cand_set in ballot.ranking:
        num_cands += len(cand_set)

        if num_cands > 6:
            break

        new_ranking.append(cand_set)

    return Ballot(ranking = new_ranking if len(new_ranking) >0 else None)

pp = clean_profile(pp, clean_ballot_func= remove_overvotes)


In [16]:
pp.to_pickle("Portland_D1_overvotes_as_ties_pp.pkl")

In [23]:
from votekit.elections import Borda

overvotes_as_ties_pp = PreferenceProfile.from_pickle("Portland_D1_overvotes_as_ties_pp.pkl")
overvotes_removed_pp = PreferenceProfile.from_pickle("../data/Portland_D1_cleaned_votekit_pref_profile.pkl")

e_ties = Borda(overvotes_as_ties_pp, m=3)
e_removed = Borda(overvotes_removed_pp, m=3)


In [24]:
print(e_ties.get_ranking())
print(e_removed.get_ranking())

(frozenset({'Candace Avalos'}), frozenset({'Steph Routh'}), frozenset({'Loretta Smith'}), frozenset({'Jamie Dunphy'}), frozenset({'Timur Ender'}), frozenset({'Terrence Hayes'}), frozenset({'Noah Ernst'}), frozenset({'Cayle Tern'}), frozenset({'Doug Clove'}), frozenset({'David Linn'}), frozenset({'Michael (Mike) Sands'}), frozenset({'Joe Allen'}), frozenset({'Deian Salazar'}), frozenset({'Peggy Sue Owens'}), frozenset({'Joe Furi'}), frozenset({'Thomas Shervey'}), frozenset({'Uncertified Write In'}))
(frozenset({'Candace Avalos'}), frozenset({'Steph Routh'}), frozenset({'Loretta Smith'}), frozenset({'Jamie Dunphy'}), frozenset({'Timur Ender'}), frozenset({'Terrence Hayes'}), frozenset({'Noah Ernst'}), frozenset({'Cayle Tern'}), frozenset({'Doug Clove'}), frozenset({'David Linn'}), frozenset({'Michael (Mike) Sands'}), frozenset({'Joe Allen'}), frozenset({'Deian Salazar'}), frozenset({'Peggy Sue Owens'}), frozenset({'Joe Furi'}), frozenset({'Thomas Shervey'}), frozenset({'Uncertified Write